Library Imports


In [13]:
import cv2
import numpy as np
import os
from pathlib import Path

# Path to the folder containing your script
script_dir = os.getcwd() # 

# Build the path relative to that folder
address2 = os.path.join(script_dir, "Raw Images", "cam0_capture_2_20250828_054449.jpg")
address1 = os.path.join(script_dir, "Raw Images", "cam1_capture_2_20250828_054449.jpg")

img1 = cv2.imread(address1)
img2 = cv2.imread(address2)



2 Frames Alignment Function 


In [ ]:
# This script aligns two images using feature-based methods.
# It uses ORB feature detection and matching to compute a homography matrix,   
# which is then used to warp one image to align with the other.
# The aligned image is then tested against the original image to ensure alignment.


## 03.09.2025 22:16: Script doesn't align properly lowlight images, and saves in the wrong folder


def align_images_feature_based(img1, img2):
    """
    Align img2 to img1 using ORB feature matching + homography.
    Handles both grayscale and color images.
    """

    # Ensure grayscale conversion only if needed
    if len(img1.shape) == 3:  # color
        gray1 = cv2.equalizeHist(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY))
    else:  # already grayscale
        gray1 = cv2.equalizeHist(img1)

    if len(img2.shape) == 3:  # color
        gray2 = cv2.equalizeHist(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY))
    else:  # already grayscale
        gray2 = cv2.equalizeHist(img2)

    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(5000)
    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    if len(matches) < 4:
        raise ValueError("Not enough matches to compute homography.")

    matches = sorted(matches, key=lambda x: x.distance)

    # Extract location of good matches
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate the homography matrix
    M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

    # Warp img2 to align with img1 (keep same size as img1)
    aligned = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))
    
    inliers = mask.sum() if mask is not None else 0

    return aligned, inliers


def align_images_with_mirror_check(img1, img2):
    best_aligned = None
    best_inliers = -1

    # Try normal alignment
    # try:
    #     aligned, inliers = align_images_feature_based(img1, img2)
    #     if inliers > best_inliers:
    #         best_inliers = inliers
    #         best_aligned = aligned
    # except Exception as e:
    #     print("Normal alignment failed:", e)

    # Try mirrored alignment
    img2_flipped = cv2.flip(img2, 1)
    try:
        aligned, inliers = align_images_feature_based(img1, img2_flipped)
        if inliers > best_inliers:
            best_inliers = inliers
            best_aligned = aligned
    except Exception as e:
        print("Mirrored alignment failed:", e)

    # if best_aligned is None:
        raise ValueError("Failed to align both normal and mirrored cases.")

    return best_aligned


def overlay_images_rgb(img1, img2_aligned, alpha=0.5):
    # Resize img2_aligned to match img1 if needed
    if img1.shape != img2_aligned.shape:
        img2_aligned = cv2.resize(img2_aligned, (img1.shape[1], img1.shape[0]))

    # Blend images using alpha
    blended = cv2.addWeighted(img1, alpha, img2_aligned, 1 - alpha, 0)
    return blended



# Align img2 to img1
aligned = align_images_with_mirror_check(img1, img2)

# Overlay for visual verification
blended = overlay_images_rgb(img1, aligned)

# Create folder if it doesn't exist
output_folder = "Aligned and Blended images"
os.makedirs(output_folder, exist_ok=True)

# Split into folder, base filename, and extension
base2 = os.path.splitext(os.path.basename(address2))[0]   # cam0_capture_1_20250828_051554
ext2 = os.path.splitext(address2)[1]                      # .jpg
 
# Build new paths
aligned_filename = os.path.join(output_folder, f"{base2}_aligned{ext2}")
blended_filename = os.path.join(output_folder, f"{base2}_blended{ext2}")

cv2.imwrite(aligned_filename, aligned)
cv2.imwrite(blended_filename, blended)

print(f"Saved aligned image to: {aligned_filename}")
print(f"Saved blended image to: {blended_filename}")


Saved aligned image to: Aligned and Blended images\cam0_capture_2_20250828_054449_aligned.jpg
Saved blended image to: Aligned and Blended images\cam0_capture_2_20250828_054449_blended.jpg


2 Frames Focus Assessment Function 

In [10]:

def focus_loss(image):
    # If already grayscale, skip conversion
    if len(image.shape) == 2:  # single channel
        gray = image
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    variance = laplacian.var()
    return variance

loss1 = focus_loss(img1)
loss2 = focus_loss(img2)
print(f"Focus loss for img1: {loss1}, img2: {loss2}")
focus_threshold = 0.5  # Example threshold for focus quality
if np.abs(loss1 - loss2) < focus_threshold:
    print("The images are in focus.")
else:
    print("The images are not in focus.")

AttributeError: 'NoneType' object has no attribute 'shape'

Video Alignment Function


In [ ]:
import os
import subprocess
import cv2

# ---------------------------
# Helper function to convert & rotate raw H.264
# ---------------------------
def remux_and_rotate_h264(input_file, output_file, fps=25, rotation=None):
    """
    Convert raw H.264 to MP4 with optional rotation (re-encoding required).
    rotation: None, 90, 180, 270 (degrees clockwise)
    """
    if os.path.exists(output_file):
        print(f"MP4 already exists: {output_file}")
        return

    cmd = ["ffmpeg", "-y", "-framerate", str(fps), "-i", input_file]

    # Apply rotation with ffmpeg filters
    if rotation == 90:
        cmd += ["-vf", "transpose=1"]  # 90° clockwise
    elif rotation == 180:
        cmd += ["-vf", "transpose=1,transpose=1"]  # 180° (two 90° clockwise)
    elif rotation == 270:
        cmd += ["-vf", "transpose=2"]  # 90° counter-clockwise (270° clockwise)

    # Re-encode with libx264 to apply rotation
    cmd += ["-c:v", "libx264", "-pix_fmt", "yuv420p", output_file]

    subprocess.run(cmd, check=True)
    print(f"Created MP4: {output_file}")

# ---------------------------
# File paths
# ---------------------------
cam0_h264 = "cam0 record.h264"
cam1_h264 = "cam1 record.h264"
cam0_mp4 = "cam0 record.mp4"
cam1_mp4 = "cam1 record.mp4"
output_aligned = "cam1 record_aligned.mp4"

# ---------------------------
# Step 1: Convert and rotate raw H.264 to MP4
# ---------------------------
remux_and_rotate_h264(cam0_h264, cam0_mp4, fps=25, rotation=180)
remux_and_rotate_h264(cam1_h264, cam1_mp4, fps=25, rotation=90)

# ---------------------------
# Step 2: Open MP4s with OpenCV
# ---------------------------
cap0 = cv2.VideoCapture(cam0_mp4)  # reference
cap1 = cv2.VideoCapture(cam1_mp4)  # to align

ret0, frame_ref = cap0.read()
ret1, frame_target = cap1.read()
if not (ret0 and ret1):
    raise ValueError("Failed to read the first frames from the videos.")

h, w = frame_ref.shape[:2]
fps = cap0.get(cv2.CAP_PROP_FPS) or 25
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_aligned, fourcc, fps, (w, h))

# Reset to first frame
cap0.set(cv2.CAP_PROP_POS_FRAMES, 0)
cap1.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ---------------------------
# Step 3: Frame-by-frame alignment
# ---------------------------
while True:
    ret0, frame_ref = cap0.read()
    ret1, frame_target = cap1.read()
    if not (ret0 and ret1):
        break

    # Align cam1 to cam0
    aligned_frame = align_images_with_mirror_check(frame_ref, frame_target)
    out.write(aligned_frame)

# ---------------------------
# Step 4: Cleanup
# ---------------------------
cap0.release()
cap1.release()
out.release()

print("Alignment complete. Output saved to:", output_aligned)


MP4 already exists: cam0 record.mp4
MP4 already exists: cam1 record.mp4
